# s-d model + tight binding model, cluster ansatz by Yordanov

## Calculation speed

1. 3 nodes, 3 electrons

   Single PC, Intel core i7

   __Yordanov ansatz: 1000 iteration, 110 sec__

   Even if I staart with correct solution, it takes 55 sec to finish simulations. In a first iteration it jumps of the solution and then slowly returns back.

    __"Classical" cluster ansatz: 1000 iterations, 325 sec__

   3 times longer than Yordanov ansatz, which is in agreement to 3 times longer quantum circuit for classical ansatz.

   __Problem agnostic short anzatz (2 layers of rotations and entagling) with particle conservation in the cost function: 1000 iterations, 15 sec__

   This ansatz is not reliable, does not give correct solution all the time. Depends on initial parameters values.

   

   

In [1]:
#Qiskit modules
import qiskit
from qiskit import QuantumRegister as Q_R
from qiskit import ClassicalRegister as C_R
from qiskit_aer import Aer
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit.primitives import StatevectorSampler, StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

#math modules
import math
import numpy as np

# SciPy minimizer routine
from scipy.optimize import minimize

import time

#My libraries
import vqe_funcs

In [2]:
PI = np.pi

## Here are the functions performing Jordan-Wigner transformation for fermionc Pauli operators and kinetci energy of electrons 

Important notes:

1) This is spin-depenedent version, so each node has spin-coordinate and is described by 2 qubits.

   Spin-up state for node i is described by qubit i.

   Spin-down state for node i is described by qubit i + N, where N is the nodes number.

In [3]:
from vqe_funcs import sigma_x, sigma_y, sigma_z, kinetic_energy, full_ham

## Magnetization distribution description

In [4]:
#Uniform magnetization along z axis
#nodes_number = 4
#mag = []

def uniform_z(nodes_number):
    mag = []
    for i in range(nodes_number):
        mag.append([0, 0, 1])
    return mag

## Full Hamiltonian

The Hamiltonian includes kinteic energy and s-d exchage energy

In [5]:
#nodes_number = 2
#mag = uniform_z(nodes_number)
#full_ham(nodes_number, mag, 4, 1, periodic = True)

In [6]:
def anzatz_qc(theta, nodes_number, electrons_number):
    qubits_number = 2 * nodes_number
    q_r = Q_R(qubits_number)
    v_qc = QuantumCircuit(q_r)
    for i in range(electrons_number):
        v_qc.x(i)
    v_qc = vqe_funcs.exc_yordanov(qubits_number, theta, v_qc)
    return v_qc

In [7]:

#nodes_number = 6
#electrons_number = 2
#n_qubits = nodes_number * 2
#len_se = n_qubits * (n_qubits - 1) / 2
#len_de = n_qubits * (n_qubits - 1) * (n_qubits - 2) * (n_qubits - 3) / 4 / 2
#len_tot = int(len_se + len_de)

#theta = [0]
#for i in range(len_tot):
#    theta.append(0)

#theta[8] = PI / 9

#qc = anzatz_qc(theta, nodes_number, electrons_number)


In [8]:
#qc.decompose().count_ops()

In [9]:
# defining a cost function
cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}
def estim(theta, nodes_number, hamiltonian, electrons_number):
    v_qc = anzatz_qc(theta, nodes_number, electrons_number)
    
    estimator = StatevectorEstimator()
    job = estimator.run([(v_qc, hamiltonian)])
    estimator_expvals = job.result()[0].data.evs
    
      
    #print 
    #print('iter: ' + str( cost_history_dict["iters"]) + ', Energy: ' + str(estimator_expvals))
    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = theta
    cost_history_dict["cost_history"].append(estimator_expvals)
    return estimator_expvals

In [10]:
# Hamiltonian parameters
nodes_number = 3 #number of nodes
n_qubits = nodes_number * 2
J = -5 #s-d exchange constant
t = 2 #hopping matrix element (kinetic energy coefficient)
electrons_number = 3

#Cost history initialization
cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

import random
num = random.random()
#print(num)


mag = []
mag = uniform_z(nodes_number)

#initialize the state
len_se = n_qubits * (n_qubits - 1) / 2
len_de = n_qubits * (n_qubits - 1) * (n_qubits - 2) * (n_qubits - 3) / 4 / 2
len_tot = int(len_se + len_de)
x0 = []
for i in range(len_tot):
    x0.append(0)
    #x0.append(random.random())

#define the Hamiltonian
hamiltonian = full_ham(nodes_number, mag, J, t, periodic = True)
#print(hamiltonian)
#Optimization
start = time.time()
res = minimize(
        estim,
        x0,
        args=(nodes_number , hamiltonian, electrons_number),
        method="cobyla",
    )
end = time.time()
print(['time elapsed: ' + str(end - start) + ' sec'])
#print(res)
#print(cost_history_dict)

['time elapsed: 76.63820552825928 sec']


In [93]:
print(res)
print(cost_history_dict)

 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: -14.999999961112941
       x: [ 1.000e+00  1.389e-05 ...  4.667e-11  4.217e-11]
    nfev: 544
   maxcv: 0.0
{'prev_vector': array([ 9.99986111e-01,  1.38885356e-05,  2.93393659e-05, -1.38629451e-05,
        6.69744359e-06,  4.88303147e-06,  9.96621513e-06,  8.75897339e-06,
        1.13410222e-05, -8.95917976e-06,  4.15077680e-05, -3.28715203e-05,
        3.37941523e-07,  3.37944855e-07,  1.00000034e+00,  4.15275771e-11,
        4.45337655e-11,  6.09817730e-11,  1.11987619e-11,  5.36187682e-11,
        4.85056644e-11,  2.13171898e-08,  2.89351069e-05, -2.78913944e-05,
       -2.35560384e-05, -2.20917741e-05, -3.43130262e-05,  2.39891722e-11,
       -2.04447623e-11,  4.70272239e-11,  4.06608782e-11, -7.38343606e-11,
        3.06718943e-11,  3.73868384e-11, -4.65374632e-11,  7.00077203e-11,
        4.82926297e-11,  4.40399586e-11,  4.64075826e-11,  4.82587011e-11,
        4.78839189e-11,  2.97794561e-11, 

In [94]:
q_r = Q_R(nodes_number * 2)
cl_r = C_R(nodes_number * 2)
qc_f = QuantumCircuit(q_r,cl_r)
qc_1 = anzatz_qc(res.x, nodes_number, electrons_number)
qc_f.append(qc_1, q_r)

#for i in range(bit_size):
#    qc_f.measure(i,i)
SimulatorAer = AerSimulator()

qc_f.save_statevector()

#qc_f.decompose().draw('mpl')

In [95]:
#qc_f.decompose().count_ops()

In [96]:
from qiskit.quantum_info import partial_trace, DensityMatrix
from qiskit.visualization import plot_state_city

circ = transpile(qc_f, backend = SimulatorAer)
result = SimulatorAer.run(circ,shots = 1).result()
ground_state = result.get_statevector(circ)

In [97]:
import sys
sys.path.insert(0, 'C:/Users/Oleg/Google Диск/QC/Codes/QC-qiskit-codes/Library')
import aux_func as af
n_nonzero = 0
n_states = pow(2, 2 * nodes_number)
states = []
prob = []
for i in range(n_states):
    pr = pow(abs(ground_state[i]), 2)
    if pr>1/n_states:
        n_nonzero = n_nonzero + 1
        states.append(i)
        prob.append(pr)
        print('state: ' + str(af.int_2_bin_word(i, 2 * nodes_number)) + ', prob: ' + str(pr) + ', complex amplitude: ' + str(ground_state[i]))
    
print('Energy: ' + str(res.fun))

state: 000111, prob: 0.9999999976025344, complex amplitude: (0.7071067803389157-0.7071067803389152j)
Energy: -14.999999961112941
